# How to...clean up and validate banking IDs

This notebook shows how to use the **BankingIDCleaner** class to validate ID's such as LEI, ISIN and SEDOL. You can use this class in three different ways:
1. [to validate single values of text as ID's](#validate_id)
2. [to clean & validate single values of text as ID's](#clean_validate_id)
3. [to clean & validate IDs on tabular dataframes](#df)

No matter which approach you choose, you will need to import and create an object based on the **BankingIdCleaner()** class which is available in the **financial_entity_cleaner.id** package. This notebook shows how you can customize the behaviour of this class to adapt the cleaning to your own needs.   

In [1]:
# Sets up the location of the financial-entity-cleaner library relative to this notebook 
import sys
sys.path.append('../../')

In [2]:
# Import BankingIdCleaner
from financial_entity_cleaner.id import BankingIdCleaner

In [3]:
# Create an object based on the BankingIdCleaner() class
id_cleaner_obj = BankingIdCleaner()

To see all the supported ID types:

In [4]:
# Check the ID's supported by the library
id_cleaner_obj.get_types()

['lei', 'isin', 'sedol']

<div class="alert alert-block alert-danger">
<b>EXCEPTION:</b> The library throws an exception if the ID type is not supported.
</div>

In [5]:
id_cleaner_obj.id_type='test'

TypeOfBankingIdNotSupported: Financial-Entity-Cleaner (Error) <BankingIdCleaner> - The ID type(s) <test> is/are not supported.

## 1. Validating single values of text as ID's <a id="validate_id"></a>

Use **is_valid()** method to verify if an ID is valid. This methods will return:
- None if the value is not a string or has no characters in it.
- True if the value is a valid ID of the specified type
- False if the value is not a valid ID of the specified type

By default, the library assumes that the value passed as parameter is an ISIN code. 

In [6]:
# Checking the default type
print(id_cleaner_obj.id_type)

isin


In [7]:
# Testing an invalid ISIN code
print(id_cleaner_obj.is_valid('tttt0B1YW4409'))

False


In [8]:
# Testing a valid LEI code
id_cleaner_obj.id_type='lei'
print(id_cleaner_obj.is_valid('969500DPKGC9JE9F0820'))

True


In [9]:
# Testing a valid SEDOL code
id_cleaner_obj.id_type='sedol'
print(id_cleaner_obj.is_valid('2595708'))

True


The **operation mode** of **BankingIdCleaner()** class is set by **default** to be in **SILENT_MODE**, meaning that if the ID is not a string or is empty, **None** is returned instead of an error. 

In [10]:
# Testing to clean up a value that is not a string
print(id_cleaner_obj.is_valid(12345))

None


The code below shows how to change the operation mode to throw an error for invalid ID's.

In [11]:
id_cleaner_obj.mode = BankingIdCleaner.EXCEPTION_MODE

<div class="alert alert-block alert-danger">
<b>EXCEPTION MODE:</b> Now, the code below will throw a customized exception because the parameter of is_valid() method is not a string.
</div>

In [12]:
# Testing to clean up a value that is not a string
print(id_cleaner_obj.is_valid(12345))

BankingIdIsNotAString: Financial-Entity-Cleaner (Error) <BankingIdCleaner> - The input data <12345> is not a string.

In [13]:
# Back to SILENT mode
id_cleaner_obj.mode = BankingIdCleaner.SILENT_MODE

## 2. Cleaning and validating single values of text as ID's <a id="clean_validate_id"></a>

The library can also be used for cleaning and validation at the same time. In this case, it will return a dictionary with the cleaning ID and if it is valid or not:

In [14]:
id_cleaner_obj.id_type='lei'

In [15]:
# Cleaning a valid LEI code
clean_lei = id_cleaner_obj.clean('969500  dpKGC9JE9F0820')
clean_lei

{'cleaned_id': '969500DPKGC9JE9F0820', 'isvalid_id': True}

You may want to change the dictionary keys returned by the clean() method. Instead of calling them 'cleaned_id' and 'isvalid_id' you can define other names by changing the properties **output_cleaned_id** and **output_validated_id**:

In [16]:
id_cleaner_obj.output_cleaned_id = 'LEI'
id_cleaner_obj.output_validated_id = 'IS_VALID'
clean_lei = id_cleaner_obj.clean('969500DPKGC9JE9F0820')
clean_lei

{'LEI': '969500DPKGC9JE9F0820', 'IS_VALID': True}

You can reset these output names any time by calling the **reset_output_names()**: 

In [17]:
id_cleaner_obj.reset_output_names()

In [18]:
clean_lei = id_cleaner_obj.clean('969500DPKGC9JE9F0820')
clean_lei

{'cleaned_id': '969500DPKGC9JE9F0820', 'isvalid_id': True}

You may also want to return NaN if the ID is invalid. For this, set the property **invalid_ids_as_nan** to **True**. By default, it is set to False and, therefore, the clean() method will always return the ID text:

In [19]:
id_cleaner_obj.invalid_ids_as_nan = True
clean_lei = id_cleaner_obj.clean('96XX00DPKGC9JE9F0820')
clean_lei

{'cleaned_id': nan, 'isvalid_id': False}

## 3. Cleaning and validating IDs on tabular dataframe <a id="df"></a>

A more realistic scenario is to have your data in a tabular format and you are already using the Pandas library to make operations on it. You can write your own code to iterate over your pandas dataframe structure by using the clean() method to clean up some columns. However, the financial-entity-cleaner makes this task easier for you. The BankingIdCleaner() class provides the **clean_df()** method to perform the normalization of ids defined as dataframe columns. See the code below on how to apply this method: 

In [20]:
import pandas as pd

In [21]:
# Read a .csv file as a pandas dataframe object
input_filename = '../../tests/data/test_cleaner.csv'
df_original = pd.read_csv(input_filename,sep=';',encoding='utf-8', usecols=['NAME','LEI', 'ISIN', 'SEDOL'])
df_original

,NAME,LEI,ISIN,SEDOL
0,Bechel *Australia (Services) Pty Ltd,98 4500 e8da1de9a0d939,NaN,NaN
1,NRI - KELLY's MERCHANDISE (AUST) PTY LTD,NaN,au0000036949,NaN
2,meo - serviços de comunicação e multimedia SA...,5493001MT6YISZH3YV05,4ABL286115,NaN
3,"Glass Coatings & Concepts ""CBG"" LLC",34123456,4AAAA255044,NaN
4,"Brault Loisirs, Orl. SARL",NaN,FR0000036196,552111809
5,"Cole & Brothers Fabric, Services LLC.",213800UHTTV6EGY74X82,NaN,NaN
6,StarCOM Group Servizi **CAT** SRL,NaN,NaN,NaN
7,Wolbeck (Archer Daniels) *Unified* GmbH,NaN,NaN,HPA 706689
8,"Anheuser-BUSCH, Brothers (food services), LLC",US4567899888,4567788,NaN
9,"Susamar-Patino, colectores (adm) SA",NaN,ES0176252718,A7830 4516


Another important property of the BankingIdCleaner() class is the **letter_case** that defines if the output will be returned in lower, upper or title case. By default, the letter case is set to be 'lower'. If you want to change that, just set the letter_case property to **'lower'**, **'upper'** or **'title'** or use the built-in class constants LOWER_LETTER_CASE, UPPER_LETTER_CASE, TITLE_LETTER_CASE, as shown below:

In [22]:
# Set up the resultant letter case
id_cleaner_obj.output_lettercase = BankingIdCleaner.LOWER_LETTER_CASE

In [23]:
id_cleaner_obj.id_type='lei'

In [24]:
df_cleaner = id_cleaner_obj.clean_df(df_original, cols=['LEI'])
df_cleaner

Column [LEI] Type [lei] : 100%|██████████████████████████████████████████████████| 11/11 [00:00<00:00, 366.87it/s]


,NAME,LEI,ISIN,SEDOL,LEI_cleaned_id,LEI_isvalid_id
0,Bechel *Australia (Services) Pty Ltd,98 4500 e8da1de9a0d939,NaN,NaN,984500E8DA1DE9A0D939,True
1,NRI - KELLY's MERCHANDISE (AUST) PTY LTD,NaN,au0000036949,NaN,NaN,True
2,meo - serviços de comunicação e multimedia SA...,5493001MT6YISZH3YV05,4ABL286115,NaN,5493001MT6YISZH3YV05,True
3,"Glass Coatings & Concepts ""CBG"" LLC",34123456,4AAAA255044,NaN,NaN,False
4,"Brault Loisirs, Orl. SARL",NaN,FR0000036196,552111809,NaN,True
5,"Cole & Brothers Fabric, Services LLC.",213800UHTTV6EGY74X82,NaN,NaN,213800UHTTV6EGY74X82,True
6,StarCOM Group Servizi **CAT** SRL,NaN,NaN,NaN,NaN,True
7,Wolbeck (Archer Daniels) *Unified* GmbH,NaN,NaN,HPA 706689,NaN,True
8,"Anheuser-BUSCH, Brothers (food services), LLC",US4567899888,4567788,NaN,NaN,False
9,"Susamar-Patino, colectores (adm) SA",NaN,ES0176252718,A7830 4516,NaN,True


In the first row you can notice that the cleaning processing did a good job on the LEI ID. However, keep in mind that the cleaning performed by **BankingIdCleaner()** is very simple and just remove extra spaces and unicode characters. If you need to perform a more advanced cleaning task, use the **SimpleCleaner()** class provided in the financial-entity-cleaner.text package that is able to apply different pre-defined regex rules on texts or string attributes.  

The result above shows that two new columns were created with a suffix defined by the properties **output_cleaned_id** and **output_validated_id**. It also preserved the original LEI column. But, what if we want to clean and validate more than one column in the dataframe, use a prefix standard to rename the new columns and remove the original ones? Notice that we did not inform the ID type of the LEI column. If the argument **types** is not passed, the cleaning method assumes that all columns are of the same type as defined by the property **id_type**. The code below performs the cleaning on different IDs: 

In [25]:
id_cleaner_obj.output_cleaned_id = 'CLEANED'
id_cleaner_obj.output_validated_id = 'IS_VALID'
df_cleaner = id_cleaner_obj.clean_df(df_original, cols=['LEI', 'ISIN', 'SEDOL'], 
                                     remove_cols= True, 
                                     output_names_as= 'prefix',
                                     types = ['lei', 'isin', 'sedol'])
df_cleaner

Column [SEDOL] Type [sedol] : 100%|██████████████████████████████████████████████████| 11/11 [00:00<00:00, 129.48it/s]


,NAME,CLEANED_LEI,IS_VALID_LEI,CLEANED_ISIN,IS_VALID_ISIN,CLEANED_SEDOL,IS_VALID_SEDOL
0,Bechel *Australia (Services) Pty Ltd,984500E8DA1DE9A0D939,True,NaN,True,NaN,True
1,NRI - KELLY's MERCHANDISE (AUST) PTY LTD,NaN,True,AU0000036949,True,NaN,True
2,meo - serviços de comunicação e multimedia SA...,5493001MT6YISZH3YV05,True,NaN,False,NaN,True
3,"Glass Coatings & Concepts ""CBG"" LLC",NaN,False,NaN,False,NaN,True
4,"Brault Loisirs, Orl. SARL",NaN,True,FR0000036196,True,NaN,False
5,"Cole & Brothers Fabric, Services LLC.",213800UHTTV6EGY74X82,True,NaN,True,NaN,True
6,StarCOM Group Servizi **CAT** SRL,NaN,True,NaN,True,NaN,True
7,Wolbeck (Archer Daniels) *Unified* GmbH,NaN,True,NaN,True,NaN,False
8,"Anheuser-BUSCH, Brothers (food services), LLC",NaN,False,NaN,False,NaN,True
9,"Susamar-Patino, colectores (adm) SA",NaN,True,ES0176252718,True,NaN,False


All the results above set the ID to NaN if the ID is invalid (see row 4 and 9). This happens when **invalid_ids_as_nan** property is True. Therefore, make sure to set it to False if you don't want this behaviour: 

In [26]:
id_cleaner_obj.invalid_ids_as_nan = False 

In [27]:
df_cleaner = id_cleaner_obj.clean_df(df_original, cols=['LEI', 'ISIN', 'SEDOL'], 
                                     remove_cols= True, 
                                     output_names_as= 'prefix',
                                     types = ['lei', 'isin', 'sedol'])
df_cleaner

Column [SEDOL] Type [sedol] : 100%|██████████████████████████████████████████████████| 11/11 [00:00<00:00, 127.98it/s]


,NAME,CLEANED_LEI,IS_VALID_LEI,CLEANED_ISIN,IS_VALID_ISIN,CLEANED_SEDOL,IS_VALID_SEDOL
0,Bechel *Australia (Services) Pty Ltd,984500E8DA1DE9A0D939,True,NaN,True,NaN,True
1,NRI - KELLY's MERCHANDISE (AUST) PTY LTD,NaN,True,AU0000036949,True,NaN,True
2,meo - serviços de comunicação e multimedia SA...,5493001MT6YISZH3YV05,True,4ABL286115,False,NaN,True
3,"Glass Coatings & Concepts ""CBG"" LLC",34123456,False,4AAAA255044,False,NaN,True
4,"Brault Loisirs, Orl. SARL",NaN,True,FR0000036196,True,552111809,False
5,"Cole & Brothers Fabric, Services LLC.",213800UHTTV6EGY74X82,True,NaN,True,NaN,True
6,StarCOM Group Servizi **CAT** SRL,NaN,True,NaN,True,NaN,True
7,Wolbeck (Archer Daniels) *Unified* GmbH,NaN,True,NaN,True,HPA706689,False
8,"Anheuser-BUSCH, Brothers (food services), LLC",US4567899888,False,4567788,False,NaN,True
9,"Susamar-Patino, colectores (adm) SA",NaN,True,ES0176252718,True,A78304516,False


By default, the results of the ID validation are a boolean attributes, as shown below:

In [28]:
df_cleaner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   NAME            11 non-null     object
 1   CLEANED_LEI     5 non-null      object
 2   IS_VALID_LEI    11 non-null     bool  
 3   CLEANED_ISIN    7 non-null      object
 4   IS_VALID_ISIN   11 non-null     bool  
 5   CLEANED_SEDOL   3 non-null      object
 6   IS_VALID_SEDOL  11 non-null     bool  
dtypes: bool(3), object(4)
memory usage: 513.0+ bytes


Sometimes, it is necessary to have boolean values described as categorical data (0 and 1's). For example, when we want to store data in a database or perform machine learning operations on it. In order to return categorical data as the result of the validation process, you must set the property **validation_as_categorical** as **True**: 

In [29]:
id_cleaner_obj.validation_as_categorical = True 

In [30]:
df_cleaner = id_cleaner_obj.clean_df(df_original, cols=['LEI', 'ISIN', 'SEDOL'], 
                                     remove_cols= True, 
                                     output_names_as= 'prefix',
                                     types = ['lei', 'isin', 'sedol'])
df_cleaner

Column [SEDOL] Type [sedol] : 100%|██████████████████████████████████████████████████| 11/11 [00:00<00:00, 117.09it/s]


,NAME,CLEANED_LEI,IS_VALID_LEI,CLEANED_ISIN,IS_VALID_ISIN,CLEANED_SEDOL,IS_VALID_SEDOL
0,Bechel *Australia (Services) Pty Ltd,984500E8DA1DE9A0D939,1.0,NaN,NaN,NaN,NaN
1,NRI - KELLY's MERCHANDISE (AUST) PTY LTD,NaN,NaN,AU0000036949,1.0,NaN,NaN
2,meo - serviços de comunicação e multimedia SA...,5493001MT6YISZH3YV05,1.0,4ABL286115,0.0,NaN,NaN
3,"Glass Coatings & Concepts ""CBG"" LLC",34123456,0.0,4AAAA255044,0.0,NaN,NaN
4,"Brault Loisirs, Orl. SARL",NaN,NaN,FR0000036196,1.0,552111809,0.0
5,"Cole & Brothers Fabric, Services LLC.",213800UHTTV6EGY74X82,1.0,NaN,NaN,NaN,NaN
6,StarCOM Group Servizi **CAT** SRL,NaN,NaN,NaN,NaN,NaN,NaN
7,Wolbeck (Archer Daniels) *Unified* GmbH,NaN,NaN,NaN,NaN,HPA706689,0.0
8,"Anheuser-BUSCH, Brothers (food services), LLC",US4567899888,0.0,4567788,0.0,NaN,NaN
9,"Susamar-Patino, colectores (adm) SA",NaN,NaN,ES0176252718,1.0,A78304516,0.0


The validation columns were converted to float data because of the null values in some of the cells.

In [31]:
df_cleaner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NAME            11 non-null     object 
 1   CLEANED_LEI     5 non-null      object 
 2   IS_VALID_LEI    5 non-null      float64
 3   CLEANED_ISIN    7 non-null      object 
 4   IS_VALID_ISIN   7 non-null      float64
 5   CLEANED_SEDOL   3 non-null      object 
 6   IS_VALID_SEDOL  3 non-null      float64
dtypes: float64(3), object(4)
memory usage: 744.0+ bytes
